# 04_03: Cleaning weather data

In [ ]:
import math
import collections
import dataclasses
import datetime

import numpy as np
import pandas as pd
import matplotlib.pyplot as pp

In [ ]:
import getweather

In [ ]:
pasadena = getweather.getyear('PASADENA', ['TMIN', 'TMAX'], 2001)

In [ ]:
np.mean(pasadena['TMIN']), np.min(pasadena['TMIN']), np.max(pasadena['TMIN'])

In [ ]:
pasadena['TMIN']

In [ ]:
np.isnan(pasadena['TMIN'])

In [ ]:
np.sum(np.isnan(pasadena['TMIN']))

In [ ]:
np.nanmin(pasadena['TMIN']), np.nanmax(pasadena['TMAX'])

In [ ]:
pasadena['TMIN'][np.isnan(pasadena['TMIN'])] = np.nanmean(pasadena['TMIN'])
pasadena['TMAX'][np.isnan(pasadena['TMAX'])] = np.nanmean(pasadena['TMAX'])

In [ ]:
pasadena['TMIN']

In [ ]:
pp.plot(pasadena['TMIN'])

In [ ]:
xdata = np.array([0,1,4,5,7,9], 'd')
ydata = np.array([10,5,2,7,7.5,10], 'd')

pp.plot(xdata, ydata, '--o') # plot with dashed lines and thick dots

In [ ]:
xnew = np.linspace(0, 9, 19)
xnew

In [ ]:
ynew = np.interp(xnew, xdata, ydata)

In [ ]:
pp.plot(xnew, ynew, '--o')
pp.plot(xdata, ydata, '.')

In [ ]:
pasadena = getweather.getyear('PASADENA', ['TMIN', 'TMAX'], 2001)

In [ ]:
good = ~np.isnan(pasadena['TMIN']) # tilde is negation
x = np.arange(0, 365)

np.interp(x, x[good], pasadena['TMIN'][good])

In [ ]:
def fillnans(array):
    good = ~np.isnan(array)
    x = np.arange(len(array))

    return np.interp(x, x[good], array[good])

In [ ]:
pp.plot(fillnans(pasadena['TMIN']))
pp.plot(fillnans(pasadena['TMAX']))